In [10]:
import os
import scipy
import numpy as np

from PIL import Image
import cv2
import mat73

In [11]:
FORMAT = 512, 482
PATH_RES = r'data_cleaned'

num = 0

In [12]:
def add_spectral_hs_data(PATH, PATH_RES):
    file_list = os.listdir(PATH)

    for f in range(len(file_list)): 
        fpath = os.path.join(PATH, file_list[f])

        mat_file = scipy.io.loadmat(fpath)
        mat_data = mat_file['cube']  

        res = np.swapaxes(mat_data, 0, 2)
        res = np.swapaxes(res, 2, 1)

        for i in range(len(res)):
            im = Image.fromarray(res[i])
            im.save(os.path.join(PATH_RES, str(f + num) + '_' + str(i) + '_ARAD-HS.tiff'))

In [13]:
def add_rgb_hs_data(PATH, PATH_RES):
    global num
    file_list = os.listdir(PATH)

    for f in range(len(file_list)):
        im = Image.open(os.path.join(PATH, file_list[f]))
        im.save(os.path.join(PATH_RES, str(f + num) + '_ARAD-HS.png'))

    num = int(len(os.listdir(PATH_RES)) / 32)

In [14]:
add_spectral_hs_data(r'data\NTIRE2020_Train_Spectral', PATH_RES)
add_rgb_hs_data(r'data\NTIRE2020_Train_Clean', PATH_RES)

add_spectral_hs_data(r'data\NTIRE2020_Validation_Spectral', PATH_RES)
add_rgb_hs_data(r'data\NTIRE2020_Validation_Clean', PATH_RES)

In [15]:
PATH = r'data\complete_ms_data'
directory_list = os.listdir(PATH)

for d in range(len(directory_list) - 1):
    dpath = os.path.join(PATH, directory_list[d])
    dpath = os.path.join(dpath, directory_list[d])

    file_list = os.listdir(dpath)

    for f in range(len(file_list[:-2])): 
        image = cv2.imread(os.path.join(dpath, file_list[f]))
        image = image[:FORMAT[1], :FORMAT[0]]

        cv2.imwrite(os.path.join(PATH_RES, str(d + num) + '_' + str(f) + '_CAVE.tiff'), image)

    image = cv2.imread(os.path.join(dpath, file_list[-2]))
    image = image[:FORMAT[1], :FORMAT[0]]
    cv2.imwrite(os.path.join(PATH_RES, str(d + num) + '_CAVE.png'), image)

num = int(len(os.listdir(PATH_RES)) / 32)

In [16]:
def add_spectral_hs2018_data(PATH, PATH_RES):
    local_num = int(len(os.listdir(PATH_RES)) / 32)
    
    file_list = os.listdir(PATH)

    for f in range(len(file_list)): 
        fpath = os.path.join(PATH, file_list[f])

        mat_file = mat73.loadmat(fpath)
        mat_data = mat_file['rad']  

        res = np.swapaxes(mat_data, 0, 2)
        res = np.swapaxes(res, 2, 1)

        for i in range(len(res)):
            
            res[i] = cv2.normalize(res[i],  np.zeros(FORMAT), 0, 1, cv2.NORM_MINMAX)

            images = [res[i][x * FORMAT[1]: (x + 1) * FORMAT[1], y * FORMAT[0]: (y + 1) * FORMAT[0]] for x in range(2) for y in range(2)]

            for im in range(len(images)):
                image = Image.fromarray(images[im])
                image.save(os.path.join(PATH_RES, str(f + local_num + im) + '_' + str(i) + '_BGU-HS.tiff'))

        local_num += 3

In [17]:
def add_rgb_hs2018_data(PATH, PATH_RES):
    global num
    file_list = os.listdir(PATH)

    for f in range(len(file_list)):
        im = Image.open(os.path.join(PATH, file_list[f]))
        im = np.array(im)
        images = [im[x * FORMAT[1]: (x + 1) * FORMAT[1], y * FORMAT[0]: (y + 1) * FORMAT[0]] for x in range(2) for y in range(2)]

        for im in range(len(images)):
            image = Image.fromarray(images[im])
            image.save(os.path.join(PATH_RES, str(f + num  + im) + '_BGU-HS.png'))
        num += 3

    num = int(len(os.listdir(PATH_RES)) / 32)

In [18]:
add_spectral_hs2018_data(r'data\NTIRE2018_Train1_Spectral', PATH_RES)
add_rgb_hs2018_data(r'data\NTIRE2018_Train1_Clean', PATH_RES)

add_spectral_hs2018_data(r'data\NTIRE2018_Train2_Spectral', PATH_RES)
add_rgb_hs2018_data(r'data\NTIRE2018_Train2_Clean', PATH_RES)

add_spectral_hs2018_data(r'data\NTIRE2018_Validate_Spectral', PATH_RES)
add_rgb_hs2018_data(r'data\NTIRE2018_Validate_Clean', PATH_RES)